<a href="https://colab.research.google.com/github/ardalanebrahimi/EDU/blob/master/machin_vision_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
import zipfile
import os
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [1]:
drive.mount('/content/drive')
#drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with zipfile.ZipFile('/content/drive/My Drive/cifar/cifar.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/cifar')

In [0]:
def create_dirs() :
  base_dir = '/content/cifar'

  train_dir = os.path.join(base_dir,'train')
  test_dir = os.path.join(base_dir,'test')
  validation_dir = os.path.join(base_dir,'validation')

  train_cat_dir = os.path.join(train_dir,'cat')
  train_dog_dir = os.path.join(train_dir,'dog')
  train_airplane_dir = os.path.join(train_dir,'airplane')
  train_automobile_dir = os.path.join(train_dir,'automobile')
  train_bird_dir = os.path.join(train_dir,'bird')
  train_deer_dir = os.path.join(train_dir,'deer')
  train_frog_dir = os.path.join(train_dir,'frog')
  train_horse_dir = os.path.join(train_dir,'horse')
  train_ship_dir = os.path.join(train_dir,'ship')
  train_truck_dir = os.path.join(train_dir,'truck')


  validation_cat_dir = os.path.join(validation_dir,'cat')
  validation_dog_dir = os.path.join(validation_dir,'dog')
  validation_airplane_dir = os.path.join(validation_dir,'airplane')
  validation_automobile_dir = os.path.join(validation_dir,'automobile')
  validation_bird_dir = os.path.join(validation_dir,'bird')
  validation_deer_dir = os.path.join(validation_dir,'deer')
  validation_frog_dir = os.path.join(validation_dir,'frog')
  validation_horse_dir = os.path.join(validation_dir,'horse')
  validation_ship_dir = os.path.join(validation_dir,'ship')
  validation_truck_dir = os.path.join(validation_dir,'truck')

  test_cat_dir = os.path.join(test_dir,'cat')
  test_dog_dir = os.path.join(test_dir,'dog')
  test_airplane_dir = os.path.join(test_dir,'airplane')
  test_automobile_dir = os.path.join(test_dir,'automobile')
  test_bird_dir = os.path.join(test_dir,'bird')
  test_deer_dir = os.path.join(test_dir,'deer')
  test_frog_dir = os.path.join(test_dir,'frog')
  test_horse_dir = os.path.join(test_dir,'horse')
  test_ship_dir = os.path.join(test_dir,'ship')
  test_truck_dir = os.path.join(test_dir,'truck')

  ###########################################
  
def create_model():
  _model = models.Sequential()
  _model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape = (150, 150, 3)))
  _model.add(layers.MaxPooling2D(2, 2))
  _model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  _model.add(layers.MaxPooling2D(2, 2))
  _model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  _model.add(layers.MaxPooling2D(2, 2))
  _model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  _model.add(layers.Flatten())
  _model.add(layers.Dense(512, activation = 'relu'))
  _model.add(layers.Dense(10, activation = 'softmax'))

  _model.compile(loss='categorical_crossentropy',
               optimizer= 'rmsprop',
               metrics = ['accuracy'])
  return _model

##############################################
create_dirs()
##############################################
def fit_model(train_datagen = ImageDataGenerator(rescale=1./255),
             train_batch_size=20,
             _epochs = 30):
  _train_datagen = train_datagen

  test_datagen = ImageDataGenerator(rescale=1./255)

  train_generator = _train_datagen.flow_from_directory(
      train_dir,
      target_size=(150,150),
      batch_size=20,
      class_mode = 'categorical')


  validation_generator = test_datagen.flow_from_directory(
      validation_dir,
      target_size=(150,150),
      batch_size=train_batch_size,
      class_mode = 'categorical')

  _history = model.fit_generator(
      train_generator,
      steps_per_epoch = 100,
      epochs = _epochs,
      validation_data=validation_generator,
      validation_steps=50)
  
  return _history

#################################################
def train_val_by_metric(history,metric):
  history_dict = history.history
  #history_dict.keys()
  values = history_dict[metric]
  val_values = history_dict['val_'+metric]

  epochs = range(1, len(values) + 1)
  
  plt.plot(epochs, values, 'bo', label= 'Training '+metric)
  plt.plot(epochs, val_values, 'b', label= 'Validation '+metric)
  metricTitle = 'Loss' if metric == 'loss' else 'Accuracy'
  plt.title('Training and validation '+metricTitle)
  plt.xlabel('Epochs')
  plt.ylabel(metricTitle)
  plt.legend()

  plt.show()
###################################################
def train_val_res(history):
  train_val_by_metric(history,'loss')
  train_val_by_metric(history,'acc')

In [0]:
model = create_model()
model.summary()
history = fit_model()
model.save('cifar10_small.h5')
train_val_res(history)

In [36]:
model = create_model()
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
history = fit_model(train_datagen,32,100)
train_val_res(history)

Found 2500 images belonging to 10 classes.
Found 500 images belonging to 10 classes.
Epoch 1/100
100/100 [==============================] - 15s 151ms/step - loss: 14.4807 - acc: 0.1015 - val_loss: 9.8301 - val_acc: 0.0997
Epoch 2/100
100/100 [==============================] - 14s 137ms/step - loss: 14.5143 - acc: 0.0995 - val_loss: 9.7805 - val_acc: 0.0965
Epoch 3/100
100/100 [==============================] - 14s 140ms/step - loss: 14.4982 - acc: 0.1005 - val_loss: 9.7431 - val_acc: 0.1036
Epoch 4/100
100/100 [==============================] - 14s 140ms/step - loss: 14.5224 - acc: 0.0990 - val_loss: 9.7851 - val_acc: 0.1017
Epoch 5/100
100/100 [==============================] - 14s 142ms/step - loss: 14.5143 - acc: 0.0995 - val_loss: 9.7231 - val_acc: 0.0972
Epoch 6/100
100/100 [==============================] - 14s 140ms/step - loss: 14.5305 - acc: 0.0985 - val_loss: 9.8436 - val_acc: 0.0985
Epoch 7/100
100/100 [==============================] - 14s 138ms/step - loss: 14.4257 - acc: 

KeyboardInterrupt: ignored